# Pairs Trading
In this notebook, we will delve into the concept of pairs trading, understand the underlying principles, and learn how to implement and evaluate pairs trading strategies using Python. We will explore the steps involved in identifying suitable pairs, formulating trading rules, and assessing the profitability and risk of our trading strategies.

Pairs trading is based on the notion that, over time, the prices of two assets that are historically correlated tend to revert to their mean relationship. This mean reversion behavior forms the foundation of the strategy, which involves taking long and short positions in the two assets to profit from the expected convergence of their prices.

In [ ]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf

#### Data

In [ ]:
dov = yf.download('DOV','2021-01-01',datetime.today(), progress=False)['Close'].rename('Dover Corp.')
ame = yf.download('AME','2021-01-01',datetime.today(), progress=False)['Close'].rename('Ametek Inc.')

In [ ]:
pd.concat([dov, ame], axis=1).plot();

#### Spread visualization

In [ ]:
plt.figure(figsize=(10,6))
# Create two charts on the same figure.
ax1 = plt.subplot2grid((10,1), (0,0), rowspan = 4, colspan = 1)
ax2 = plt.subplot2grid((10,1), (5,0), rowspan = 4, colspan = 1)

# First chart:
# Plot the closing price on the first chart
ax1.plot(dov, linewidth=2)
ax1.plot(ame, linewidth=2)
ax1.set_title('Pair')

# Second chart
# Plot the Spread
spreads = # Complete
ax2.set_title('Spread Monitor')
ax2.plot(spreads, color='orange', linewidth=1)
# Add two horizontal lines, signalling the buy and sell ranges.
# Oversold
ax2.axhline(17, linestyle='--', linewidth=1, color='green')
# Overbought
ax2.axhline(-17, linestyle='--', linewidth=1, color='green');

#### Translate into a strategy

In [ ]:
transactions = {'date': [], 'price': [], 'transaction': [], 'firm': []}

signal = 0
for i, spread in enumerate(spreads):
    if #Complete:
        transactions['date'].append(dov.index[i])
        transactions['price'].append(dov[i])
        transactions['transaction'].append('Short')
        transactions['firm'].append('Dover')
        transactions['date'].append(ame.index[i])
        transactions['price'].append(ame[i])
        transactions['transaction'].append('Long')
        transactions['firm'].append('Ametek')
        signal = 1
    elif #Complete:
        transactions['date'].append(dov.index[i])
        transactions['price'].append(dov[i])
        transactions['transaction'].append('Long')
        transactions['firm'].append('Dover')
        transactions['date'].append(ame.index[i])
        transactions['price'].append(ame[i])
        transactions['transaction'].append('Short')
        transactions['firm'].append('Ametek')
        signal = 0
    if #Complete:
        transactions['date'].append(dov.index[i])
        transactions['price'].append(dov[i])
        transactions['transaction'].append('Long')
        transactions['firm'].append('Dover')
        transactions['date'].append(ame.index[i])
        transactions['price'].append(ame[i])
        transactions['transaction'].append('Short')
        transactions['firm'].append('Ametek')
        signal = -1
    elif #Complete:
        transactions['date'].append(dov.index[i])
        transactions['price'].append(dov[i])
        transactions['transaction'].append('Short')
        transactions['firm'].append('Dover')
        transactions['date'].append(ame.index[i])
        transactions['price'].append(ame[i])
        transactions['transaction'].append('Long')
        transactions['firm'].append('Ametek')
        signal = 0
        
transactions = pd.DataFrame(transactions)
transactions

In [ ]:
plt.figure(figsize=(10,6))

buys = transactions[transactions.transaction == 'Long'].copy()
sells = transactions[transactions.transaction == 'Short'].copy()


ax1 = plt.subplot2grid((8,1), (0,0), rowspan = 5, colspan = 1)
ax2 = plt.subplot2grid((8,1), (5,0), rowspan = 3, colspan = 1)
ax1.plot(dov, linewidth=2)
ax1.plot(ame, linewidth=2)
ax1.set_title('Pair')
ax1.plot(buys.date, buys.price, marker = '^', color = 'green', markersize = 10, label = 'Long', linewidth = 0)
ax1.plot(sells.date, sells.price, marker = 'v', color = 'r', markersize = 10, label = 'Short', linewidth = 0)
ax1.plot(closes.date, closes.price, marker = '.', color = 'orange', markersize = 10, label = 'Close', linewidth = 0)


spreads = dov-ame
ax2.set_title('Spread Monitor')
ax2.plot(spreads, color='orange', linewidth=1)
ax2.axhline(17, linestyle='--', linewidth=1, color='green')
ax2.axhline(-17, linestyle='--', linewidth=1, color='green');